In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import DBLPScholarStructuredBenchmark

benchmark = DBLPScholarStructuredBenchmark(data_dir_path="../data/")
benchmark

18:02:36 INFO:Extracting DBLP-Scholar-Structured...
18:02:36 INFO:Reading DBLP-Scholar-Structured row_dict...
18:02:37 INFO:Reading DBLP-Scholar-Structured train.csv...
18:02:37 INFO:Reading DBLP-Scholar-Structured valid.csv...
18:02:37 INFO:Reading DBLP-Scholar-Structured test.csv...


<DBLPScholarStructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/DBLP-GoogleScholar/dblp_scholar_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'authors', 'venue', 'year']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'semantic_title': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'authors': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'venue': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'semantic_venue': {
        'source_attr': 'venue',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'year': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=benchmark.row_dict)
row_numericalizer.attr_info_dict

18:02:37 INFO:For attr=title, computing actual max_str_len
18:02:37 INFO:For attr=title, using actual_max_str_len=70
18:02:38 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
18:02:42 INFO:For attr=authors, computing actual max_str_len
18:02:42 INFO:actual_max_str_len=33 must be pair to enable NN pooling. Updating to 34
18:02:42 INFO:For attr=authors, using actual_max_str_len=34
18:02:42 INFO:For attr=venue, computing actual max_str_len
18:02:42 INFO:actual_max_str_len=45 must be pair to enable NN pooling. Updating to 46
18:02:42 INFO:For attr=venue, using actual_max_str_len=46
18:02:42 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
18:02:46 INFO:For attr=year, computing actual max_str_len
18:02:46 INFO:For attr=year, using actual_max_str_len=6


{'title': NumericalizeInfo(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=70, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_title': NumericalizeInfo(source_attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 

In [11]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=20,
    row_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

18:02:46 INFO:GPU available: True, used: True
18:02:46 INFO:TPU available: None, using: 0 TPU cores
18:02:46 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(model, datamodule)

18:02:48 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 31.1 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
15.0 M    Trainable params
16.2 M    Non-trainable params
31.1 M    Total params


18:02:48 WARNING:true_pair_set has 3066 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [15]:
model.blocker_net.get_signature_weights()

{'title': 0.38869205117225647,
 'semantic_title': 0.3098513185977936,
 'authors': 0.16647721827030182,
 'venue': 0.06935282796621323,
 'semantic_venue': 0.031916845589876175,
 'year': 0.03370974212884903}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.16493095734012192,
 'valid_f1_at_0.5': 0.6770899808551372,
 'valid_f1_at_0.7': 0.9378937893789379,
 'valid_f1_at_0.9': 0.9160908193484698,
 'valid_pair_entity_ratio_at_0.3': 6.243044619422572,
 'valid_pair_entity_ratio_at_0.5': 1.0834645669291338,
 'valid_pair_entity_ratio_at_0.7': 0.6047244094488189,
 'valid_pair_entity_ratio_at_0.9': 0.5018372703412074,
 'valid_precision_at_0.3': 0.08988480618851426,
 'valid_precision_at_0.5': 0.5140503875968992,
 'valid_precision_at_0.7': 0.9045138888888888,
 'valid_precision_at_0.9': 0.9707112970711297,
 'valid_recall_at_0.3': 0.9990654205607477,
 'valid_recall_at_0.5': 0.991588785046729,
 'valid_recall_at_0.7': 0.9738317757009346,
 'valid_recall_at_0.9': 0.8672897196261682}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.1666796722590714,
  'test_f1_at_0.5': 0.6819207218820496,
  'test_f1_at_0.7': 0.9488584474885845,
  'test_f1_at_0.9': 0.9246778989098118,
  'test_pair_entity_ratio_at_0.3': 6.152435830277632,
  'test_pair_entity_ratio_at_0.5': 1.0649554740701939,
  'test_pair_entity_ratio_at_0.7': 0.5866946045049765,
  'test_pair_entity_ratio_at_0.9': 0.4965950759559979,
  'test_precision_at_0.3': 0.0909323116219668,
  'test_precision_at_0.5': 0.5204131824889326,
  'test_precision_at_0.7': 0.9276785714285715,
  'test_precision_at_0.9': 0.9841772151898734,
  'test_recall_at_0.3': 0.9981308411214953,
  'test_recall_at_0.5': 0.9887850467289719,
  'test_recall_at_0.7': 0.9710280373831776,
  'test_recall_at_0.9': 0.8719626168224299}]